In [3]:
import os
import argparse
import math
import time

import ast
from copy import copy

import pandas as pd
import numpy as np
from glob import glob
from pyproj import Proj
from shapely.ops import transform
import rasterio

import pyproj
#ultralytics.checks()
import rioxarray
from shapely.geometry import Point

import concurrent
import concurrent.futures
from concurrent.futures.thread import ThreadPoolExecutor
import cv2

In [11]:
def tile_errors(tile_path):
    errors = []
    try: 
        tile = cv2.imread(tile_path)
        print(type(tile))
        del tile
        return tile_path
    except Exception as exc:
        print("error", tile_path)


In [5]:
tile_dir ="/work/csr33/images_for_predictions/naip_tiles"
tile_paths = glob(tile_dir + "/*")  # get a list of all of the tiles in tiles directory

In [12]:
tile_path_list = []
with concurrent.futures.ThreadPoolExecutor(7) as executor:
    futures = [executor.submit(tile_errors, tile_path) for tile_path in tile_paths[0:60]]
    for future in concurrent.futures.as_completed(futures):
        #try: 
        tile_path = future.result()
        tile_path_list.append([tile_path])
        #except Exception as exc:
        #    print(type(str(exc)))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

[ WARN:0@484.627] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:0@484.878] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_3208137_sw_17_060_20191109_20200102.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'



<class 'NoneType'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [21]:

def tile_to_chip_array(tile, x, y, item_dim): #used
    """
    https://stackoverflow.com/questions/15589517/how-to-crop-an-image-in-opencv-using-python
    ##
    x: col index
    y: row index
    """
    dimensions = tile.shape[2]
    chip_img = tile[y*item_dim:y*item_dim+item_dim, x*(item_dim):x*(item_dim)+item_dim]
    #add in back space if it is the edge of an image
    if (chip_img.shape[0] != item_dim) & (chip_img.shape[1] != item_dim): #width
        #print("Incorrect Width")
        chip = np.zeros((item_dim,item_dim,dimensions), np.uint8)
        chip[0:chip_img.shape[0], 0:chip_img.shape[1]] = chip_img
        chip_img = chip
    if chip_img.shape[0] != item_dim:  #Height
        black_height = item_dim  - chip_img.shape[0] #Height
        black_width = item_dim #- chip_img.shape[1] #width
        black_img = np.zeros((black_height,black_width,  dimensions), np.uint8)
        chip_img = np.concatenate([chip_img, black_img])
    if chip_img.shape[1] != item_dim: #width
        black_height = item_dim #- chip_img.shape[0] #Height
        black_width = item_dim - chip_img.shape[1] #width
        black_img = np.zeros((black_height,black_width, dimensions), np.uint8)
        chip_img = np.concatenate([chip_img, black_img],1)
    return(chip_img)


def chip_tiles(tile_path, chips_dir, item_dim): 
        """Segment tiles into item_dim x item_dim pixel chips, preserving resolution
        """
        tile_name_wo_ext, ext = os.path.splitext(os.path.basename(tile_path))  # File name
        try:
            tile = cv2.imread(tile_path)
            tile_height, tile_width, tile_channels = tile.shape  # the size of the tile
            # divide the tile into item_dim by item_dim chips (rounding up)
            row_index = math.ceil(tile_height / item_dim)
            col_index = math.ceil(tile_width / item_dim)

            count = 0
            for y in range(0, row_index):
                for x in range(0, col_index):
                    # 
                    # specify the path to save the image
                    chip_img = tile_to_chip_array(tile, x, y, item_dim) #chip tile
                    chip_name = tile_name_wo_ext + '_' + f"{y:02}" + '_' + f"{x:02}" + '.jpg'  #
                    chips_save_path = os.path.join(chips_dir, chip_name)  # row_col.jpg                    
                    cv2.imwrite(os.path.join(chips_save_path), chip_img) # save image
                    count += 1
                    del chip_img
            return tile_name_wo_ext, count
        except Exception as exc:
            print(exc, tile_path)


            


def chip_tiles_concurrent(tile_paths, chips_dir, item_dim=640, connections=6):
    # parse html and retrieve all href urls listed
    # create the pool of worker threads
    with concurrent.futures.ThreadPoolExecutor(connections-4) as executor:
        # dispatch all download tasks to worker threads
        futures = [executor.submit(chip_tiles, tile_path, chips_dir, item_dim=item_dim) for tile_path in tile_paths]
        # report results as they become available
        for future in concurrent.futures.as_completed(futures):
            try:
                # retrieve result
                tile_name_wo_ext, count = future.result()
            except Exception as exc:
                print(exc)
                #os.remove(tile)

In [15]:
def get_args_parse():
    parser = argparse.ArgumentParser("")
    parser.add_argument("--tile_dir", default="/work/csr33/images_for_predictions/naip_tiles", type=str)
    parser.add_argument('--img_dir', type=str, default="/work/csr33/images_for_predictions/naip_imgs")
    parser.add_argument("--connections", default=6, type=int)
    parser.add_argument("--imgsz", default=640, type=int)
    parser.add_argument("--chunk_id", type=int)
    parser.add_argument("--tilename_chunks_path", default='/hpc/home/csr33/ast_object_detection/images_for_prediction/tilename_chunks.npz', type=str)
    args = parser.parse_args()
    return args
import sys
sys.argv = ['my_notebook']
args = get_args_parse()

In [23]:
tile_name_wo_ext_list = []
count_list = []
with concurrent.futures.ThreadPoolExecutor(7) as executor:
    # dispatch all download tasks to worker threads
    futures = [executor.submit(chip_tiles, tile_path, args.img_dir, item_dim=args.imgsz) for tile_path in tile_paths[0:100]]
    # report results as they become available
    for future in concurrent.futures.as_completed(futures):
        try:
            # retrieve result
            tile_name_wo_ext, count = future.result()
            tile_name_wo_ext_list.append(tile_name_wo_ext)
            count_list.append(count)
        except Exception as exc:
            print("concurrent", exc)
            #os.remove(tile)

[ WARN:6@2071.842] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:6@2072.327] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_3208137_sw_17_060_20191109_20200102.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'

[ WARN:7@2142.297] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:7@2142.297] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_3009046_nw_15_030_20211115.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'

[ WARN:8@2191.647] global g

'NoneType' object has no attribute 'shape' /work/csr33/images_for_predictions/naip_tiles/m_3208137_sw_17_060_20191109_20200102.tif
concurrent cannot unpack non-iterable NoneType object


[ WARN:10@2401.332] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:10@2401.332] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_3009046_nw_15_030_20211115.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'



'NoneType' object has no attribute 'shape' /work/csr33/images_for_predictions/naip_tiles/m_3009046_nw_15_030_20211115.tif
concurrent cannot unpack non-iterable NoneType object


[ WARN:11@2451.777] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:11@2451.778] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_3009220_ne_15_030_20211223.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'



'NoneType' object has no attribute 'shape' /work/csr33/images_for_predictions/naip_tiles/m_3009220_ne_15_030_20211223.tif
concurrent cannot unpack non-iterable NoneType object


In [10]:
len(tile_path_list)

30

In [11]:
tile_paths[0].lower().endswith('.tif')

True

[ WARN:11@4526.215] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:11@4526.215] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_3208137_sw_17_060_20191109_20200102.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'

[ WARN:12@4526.299] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:12@4526.299] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_3009046_nw_15_030_20211115.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'

[ WARN:12@4536.701] glo

In [36]:
tile_path_list

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '/work/csr33/images_for_predictions/naip_tiles/m_3208137_sw_17_060_20191109_20200102.tif',
 None,
 None,
 None,
 None,
 None,
 None,
 '/work/csr33/images_for_predictions/naip_tiles/m_3009046_nw_15_030_20211115.tif',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '/work/csr33/images_for_predictions/naip_tiles/m_3009220_ne_15_030_20211223.tif',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [32]:
t = cv2.imread('/work/csr33/images_for_predictions/naip_tiles/m_3009220_ne_15_030_20211223.tif')

[ WARN:8@4220.473] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:8@4220.473] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_3009220_ne_15_030_20211223.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'



In [33]:
t

In [46]:
import concurrent.futures
import cv2

def tile_errors(tile_path):
    errors = []
    try:
        tile = cv2.imread(tile_path)
        if tile is None:
            errors.append(tile_path)
            del tile
    except:
        errors.append(tile_path)
    return errors

errors_list = []  
with concurrent.futures.ThreadPoolExecutor(16) as executor:
    futures = [executor.submit(tile_errors, tile_path) for tile_path in tile_paths[:100]]
    for future in concurrent.futures.as_completed(futures):
        errors = future.result()
        
        errors_list.extend(errors)
print(errors_list)

[ WARN:59@3715.420] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:59@3715.433] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_3208137_sw_17_060_20191109_20200102.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'

[ WARN:60@3719.567] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:60@3719.573] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_3009046_nw_15_030_20211115.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'

[ WARN:61@3770.522] glo

['/work/csr33/images_for_predictions/naip_tiles/m_3208137_sw_17_060_20191109_20200102.tif', '/work/csr33/images_for_predictions/naip_tiles/m_3009046_nw_15_030_20211115.tif', '/work/csr33/images_for_predictions/naip_tiles/m_3009220_ne_15_030_20211223.tif']


[ WARN:11@3935.320] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:11@3935.329] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_4406908_sw_19_060_20211006.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'

[ WARN:3@3935.368] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:3@3935.369] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_4406908_sw_19_060_20211006.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'

[ WARN:62@3935.804] global grfmt_t

In [1]:
errors_list

NameError: name 'errors_list' is not defined

In [47]:
errors_list

['/work/csr33/images_for_predictions/naip_tiles/m_3208137_sw_17_060_20191109_20200102.tif',
 '/work/csr33/images_for_predictions/naip_tiles/m_3009046_nw_15_030_20211115.tif',
 '/work/csr33/images_for_predictions/naip_tiles/m_3009220_ne_15_030_20211223.tif']

[ WARN:62@4054.702] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:62@4054.703] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_2909411_sw_15_060_20201112.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'

[ WARN:17@4054.703] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ WARN:44@4054.703] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:17@4054.703] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_2909411_sw_15_060_20201112.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecifie

In [30]:
def tile_errors(tile_path):
    errors = []
    try: 
        tile = cv2.imread(tile_path)
        del tile
        return errors
    except Exception as exc:
        #print(str(type(exc)))
        print(tile_path)
        errors.extend([tile_path])
        return errors

errors_list = []
with concurrent.futures.ThreadPoolExecutor(16) as executor:
    # dispatch all download tasks to worker threads
    futures = [executor.submit(tile_errors, tile_path) for tile_path in tile_paths[:100]]
    # report results as they become available
    for future in concurrent.futures.as_completed(futures):
        # retrieve result
        errors = future.result()
        print(errors)
        errors_list.extend(errors)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


[ WARN:40@2218.442] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:40@2218.443] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_3208137_sw_17_060_20191109_20200102.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'



[]
[]
[]
[]
[]
[]


[ WARN:41@2230.102] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:41@2230.102] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_3009046_nw_15_030_20211115.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'



[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


[ WARN:40@2264.661] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:40@2264.679] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_3009220_ne_15_030_20211223.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'



[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


[ WARN:17@2307.934] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:17@2307.945] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_4307154_se_19_060_20211024.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'

[ WARN:27@2308.079] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:27@2308.079] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_4307154_se_19_060_20211024.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'

[ WARN:13@2308.284] global grfmt

[]
[]


[ WARN:42@2466.750] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:42@2466.751] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_3008742_sw_16_030_20211113.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'

[ WARN:43@2466.862] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:43@2466.862] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_3008742_sw_16_030_20211113.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'

[ WARN:12@2467.694] global grfmt

In [16]:
tile_paths.index('/work/csr33/images_for_predictions/naip_tiles/m_3009043_sw_15_030_20211120.tif')

835

[ WARN:10@1297.136] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)
[ERROR:10@1297.137] global loadsave.cpp:478 imread_ imread_('/work/csr33/images_for_predictions/naip_tiles/m_3008810_ne_16_030_20211112.tif'): can't read data: OpenCV(4.9.0) /io/opencv/modules/imgcodecs/src/grfmt_tiff.cpp:840: error: (-2:Unspecified error) OpenCV TIFF: failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer) in function 'readData'

[ WARN:14@1297.347] global grfmt_tiff.cpp:840 readData OpenCV TIFF(line 840): failed TIFFReadRGBATile(tif, x, y, (uint32*)src_buffer)


In [ ]:
import cv2
try:
    tile = cv2.imread('/work/csr33/images_for_predictions/naip_tiles/m_3008929_ne_16_030_20211119.tif')
except Exception as exc:
    (type(exc))

        